In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
!git clone https://github.com/savenow/Traffic_Camera_Tracking.git

Cloning into 'Traffic_Camera_Tracking'...
remote: Enumerating objects: 13765, done.
remote: Counting objects: 100% (13564/13564), done.
remote: Compressing objects: 100% (8287/8287), done.
remote: Total 13765 (delta 909), reused 13470 (delta 821), pack-reused 201
Receiving objects: 100% (13765/13765), 243.36 MiB | 26.58 MiB/s, done.
Resolving deltas: 100% (990/990), done.
Checking out files: 100% (18872/18872), done.


In [2]:
!pip install tqdm albumentations filterpy scikit-image lap pyyaml

     |████████████████████████████████| 177 kB 5.4 MB/s 
     |████████████████████████████████| 1.5 MB 38.8 MB/s 
     |████████████████████████████████| 631 kB 48.8 MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.6-py3-none-any.whl size=654020 sha256=a8d00bef854bd71779a6c0df4613b251b4cc719bcb0f97683c1019bfc60e7149
  Stored in directory: /root/.cache/pip/wheels/89/72/98/3ebfdba1069a9a8eaaa7ae7265cfd67d63ef0197aaee2e5f9c
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110474 sha256=89e08193c085c35a23ad57d952fbf57c0b19b39dbabd13b498dfe17c09b85398
  Stored in directory: /root/.cache/pip/wheels/ce/e0/ee/a2b3c5caab3418c1ccd8c4de573d4cbe13315d7e8b0a55fbc2
  Created wheel for lap: filename=lap-0.4.0-cp37-cp37m-linux_x86_64.whl size=1590157 sha256=e7dfa31eb4e52092dc0eb724b94aa02bfafe5211033c0c5543d88973a4786b3e
  Stored in directory: /root/.cache/pip/wheels/b1/0b/e3/ef9daf1b5547b56389e42c80c3100f1e6479bf5fd00fd9d6ba
Successfully built imgaug filterpy lap
  At

In [3]:
# download input video file and weight file
!gdown --id 1EqndRhq7FZE4C4y-h7Y-KX3btGt0gFLr
!gdown --id 1vS6YN36mBxrpqug7OPAAoIAXMqnjHyOC
!gdown --id 1TgDjntHh3THnpMvnGDL99kZHHLocHgVh
!gdown --id 1LilfAbWTEq8z40Tkjm_Kq_2CuWJ0T1UY

Downloading...
From: https://drive.google.com/uc?id=1EqndRhq7FZE4C4y-h7Y-KX3btGt0gFLr
To: /content/tl_extended_epoch_30.pt
100% 154M/154M [00:01<00:00, 154MB/s]
Downloading...
From: https://drive.google.com/uc?id=1vS6YN36mBxrpqug7OPAAoIAXMqnjHyOC
To: /content/31.mp4
100% 11.1M/11.1M [00:00<00:00, 42.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1TgDjntHh3THnpMvnGDL99kZHHLocHgVh
To: /content/tl_yolo5l6_78k_bs_3.pt
100% 154M/154M [00:00<00:00, 159MB/s]
Downloading...
From: https://drive.google.com/uc?id=1LilfAbWTEq8z40Tkjm_Kq_2CuWJ0T1UY
To: /content/tl_89k_bs24_im1408_e50.pt
100% 153M/153M [00:01<00:00, 84.3MB/s]


## Tracking Code below...

In [4]:
import matplotlib
matplotlib.use('TKAgg')

In [ ]:
import importlib
import sys

import torch
import time
import cv2
import numpy as np
import random

#from Traffic_Camera_Tracking.YoloV5.yolo_v5_main_files.utils.torch_utils import time_sync
import os
from tqdm import tqdm
import torch.backends.cudnn as cudnn
from google.colab.patches import cv2_imshow

# # Importing sort
from Traffic_Camera_Tracking.YoloV5.sort_yolo import Sort
from Traffic_Camera_Tracking.YoloV5.sort_yolo import KalmanBoxTracker
# importlib.reload(Sort)

def show_tracker_bbox(input, frame, Score_ClassIDs):
    classID_dict = {0: ("Escooter", (0, 90, 255)), 1: ("Pedestrians", (255, 90, 0)), 2: ("Cyclists", (90, 255, 0))}
    
    img = frame
    for detection, Score_ClassID in zip(input, Score_ClassIDs):
        score = Score_ClassID[0]
        classID = int(Score_ClassID[1])
        color = classID_dict[classID][1]

        # For bounding box
        #print(f"Detection: {detection}")
        tracker_id = int(detection[4])
        x1, y1, x2, y2 = detection[0:4]
        x1 = int(x1)
        x2 = int(x2)
        y1 = int(y1)
        y2 = int(y2)

        #print(f"Tracker ID: {tracker_id}, classID:{classID}")
        #print(f"X1: {x1}, Y1: {y1}, X2: {x2}, Y2: {y2}")

        img = cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        
        # For the text background
        # Finds space required by the text so that we can put a background with that amount of width.
        label_1 = f'Track_ID: {tracker_id}'
        (w1, h1), _ = cv2.getTextSize(
                label_1, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
        
        label_2 = f'{classID_dict[classID][0]} {round(score*100, 1)}%'
        (w2, h2), _ = cv2.getTextSize(
                label_2, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)

        # Prints the text.    
        img = cv2.rectangle(img, (x1, y1 - 40), (x1 + w1, y1), color, -1)
        img = cv2.rectangle(img, (x1, y1 - 20), (x1 + w2, y1), color, -1)
        text_color = (0, 0, 0)
        img = cv2.putText(img, label_1, (x1, y1 - 24),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, text_color, 1)
        img = cv2.putText(img, label_2, (x1, y1 - 5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, text_color, 1)
        del color
    return img


def show_tracker_bbox_2(input, frame):
    classID_dict = {0: ("Escooter", (0, 90, 255)), 1: ("Pedestrians", (255, 90, 0)), 2: ("Cyclists", (90, 255, 0))}
    #print(input)
    img = frame
    for detection in input: # dtection = tracker in inference function
        # For bounding box
        # print(f"Detection: {detection}")
        tracker_id = int(detection[9])
        x1, y1, x2, y2 = detection[0:4]
        x1 = int(x1)
        x2 = int(x2)
        y1 = int(y1)
        y2 = int(y2)
        score = detection[4]
        classID = detection[5]

        #print(f"Tracker ID: {tracker_id}, classID:{classID}")
        #print(f"X1: {x1}, Y1: {y1}, X2: {x2}, Y2: {y2}")

        color = classID_dict[classID][1]
        img = cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        
        # For the text background
        # Finds space required by the text so that we can put a background with that amount of width.
        label_1 = f'Track_ID: {tracker_id}'
        (w1, h1), _ = cv2.getTextSize(
                label_1, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
        
        label_2 = f'{classID_dict[classID][0]} {round(score*100, 1)}%'
        (w2, h2), _ = cv2.getTextSize(
                label_2, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)

        # Prints the text.    
        img = cv2.rectangle(img, (x1, y1 - 40), (x1 + w1, y1), color, -1)
        img = cv2.rectangle(img, (x1, y1 - 20), (x1 + w2, y1), color, -1)
        text_color = (0, 0, 0)
        img = cv2.putText(img, label_1, (x1, y1 - 24),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, text_color, 1)
        img = cv2.putText(img, label_2, (x1, y1 - 5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, text_color, 1)
        del color
    return img



class TqdmExtraFormat(tqdm):
    """Provides a `total_time` format parameter"""
    @property
    def format_dict(self):
        d = super(TqdmExtraFormat, self).format_dict
        total_time = d["elapsed"] * (d["total"] or 0) / max(d["n"], 1)
        d.update(total_time=self.format_interval(total_time) + " in total")
        return d


def video_inference(input_path, output_path, model_weights, class_name=None):
    video_capture = cv2.VideoCapture(input_path)
    width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video_capture.get(cv2.CAP_PROP_FPS))
    total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
    video_duration = total_frames / fps
    codec = 'mp4v'

    video_output = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*codec), float(fps), (width, height),)
    model = torch.hub.load(r'/content/Traffic_Camera_Tracking/YoloV5/yolo_v5_main_files',
                           'custom',
                           path=model_weights,
                           source='local')
    framecount = 1

    device = torch.device('cuda:0')
    cudnn.benchmark = True
   
    #del Object_tracker
    Object_tracker = Sort(max_age=30, min_hits=7, iou_threshold=0.15)
    Object_tracker.reset_count()

    # pred = KalmanBoxTracker.predict()

    pbar = TqdmExtraFormat(total = total_frames, desc='Inference Progress: ')
    while video_capture.isOpened():
        _, frame = video_capture.read()
        
        if _:
            # Results. Try printing or modifying this array to get different classes
            results = model(frame, size=1920)
            result = results.xyxy[0]
            
            score_class_id = []
            if len(result) > 0: 
              dets = []
              for items in result:
                # items[:5].detach().cpu().numpy()
                # print(items)
                dets.append(items[:].tolist())
                #score_class_id.append((items[4].item(), items[5].item()))
              
              dets = np.array(dets)
              # print('dets : ', dets)
              tracker = Object_tracker.update(dets)
              
            else:
              tracker = Object_tracker.update()
            # print('tracker : ',tracker)
            
           
            video_output.write(show_tracker_bbox_2(tracker, frame))
            pbar.update(framecount)

            
        else:
          break

    video_capture.release()
    video_output.release()

# Model
model_weight = r'/content/tl_yolo5l6_78k_bs_3.pt'
input_directory = r'/content/31.mp4'
output_directory = r'/content/3_pt_result.mp4'
class_name = None
video_inference(input_directory, output_directory, model_weight, class_name)


## Edited above code for velocity Estimation

In [5]:
import matplotlib
matplotlib.use('TKAgg')

In [6]:
import importlib
import sys
import math
import torch
import time
import cv2
import numpy as np
import random

#from Traffic_Camera_Tracking.YoloV5.yolo_v5_main_files.utils.torch_utils import time_sync
import os
from collections import defaultdict
from tqdm import tqdm
import torch.backends.cudnn as cudnn
from google.colab.patches import cv2_imshow

# Importing sort
from Traffic_Camera_Tracking.YoloV5.sort_yolo import Sort
from Traffic_Camera_Tracking.YoloV5.sort_yolo import KalmanBoxTracker
# importlib.reload(Sort)

"""
values for drawing the boundaries in frames...
# line_1 = [(836, 121), (813, 209)]
# line_1 = [(1277, 125), (1276, 216)]
# rec = [(36,59), (1521, 59), (1521,1003), (36, 1003)]
"""

def show_tracker_bbox_2(input, frame, track_dict, track_count):
    classID_dict = {0: ("Escooter", (0, 90, 255)), 1: ("Pedestrians", (255, 90, 0)), 2: ("Cyclists", (90, 255, 0))}
    img = frame
    for detection in input: # dtection = tracker in inference function
        tracker_id = int(detection[9])
        x1, y1, x2, y2 = detection[0:4]
        x1 = int(x1)
        x2 = int(x2)
        y1 = int(y1)
        y2 = int(y2)
        score = detection[4]
        classID = detection[5]
        cx = int((x1 + x2)/2) # center_x of bbox
        cy = int((y1 +y2)/2)  # center_y of bbox
        track_dict[tracker_id].append((cx,cy))
        track_count += 1
        
        color = classID_dict[classID][1]
        
        img = cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        img = cv2.circle(img, (cx, cy), 4, color, -1)
        
        # For the text background
        # Finds space required by the text so that we can put a background with that amount of width.
        label_1 = f'Track_ID: {tracker_id}'
        (w1, h1), _ = cv2.getTextSize(
                label_1, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
        
        label_2 = f'{classID_dict[classID][0]} {round(score*100, 1)}%'
        (w2, h2), _ = cv2.getTextSize(
                label_2, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)

        # Prints the text.    
        img = cv2.rectangle(img, (x1, y1 - 40), (x1 + w1, y1), color, -1)
        img = cv2.rectangle(img, (x1, y1 - 20), (x1 + w2, y1), color, -1)
        text_color = (0, 0, 0)
        img = cv2.putText(img, label_1, (x1, y1 - 24),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, text_color, 1)
        img = cv2.putText(img, label_2, (x1, y1 - 5),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, text_color, 1)
        del color
        
        
    return img, track_dict, track_count



class TqdmExtraFormat(tqdm):
    """Provides a `total_time` format parameter"""
    @property
    def format_dict(self):
        d = super(TqdmExtraFormat, self).format_dict
        total_time = d["elapsed"] * (d["total"] or 0) / max(d["n"], 1)
        d.update(total_time=self.format_interval(total_time) + " in total")
        return d


def video_inference(input_path, output_path, model_weights, class_name=None):
    video_capture = cv2.VideoCapture(input_path)
    width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video_capture.get(cv2.CAP_PROP_FPS))
    total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
    video_duration = total_frames / fps
    codec = 'mp4v'

    video_output = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*codec), float(fps), (width, height),)
    model = torch.hub.load(r'/content/Traffic_Camera_Tracking/YoloV5/yolo_v5_main_files',
                           'custom',
                           path=model_weights,
                           source='local')
    framecount = 1

    device = torch.device('cuda:0')
    cudnn.benchmark = True
   
    #del Object_tracker
    Object_tracker = Sort(max_age=30, min_hits=7, iou_threshold=0.15)
    Object_tracker.reset_count()

    # pred = KalmanBoxTracker.predict()

    # Tracking vehicles:
    velocity_frame_window = 5
    FPS_OF_RECORDING = 30.
    track_dict = defaultdict(list)
    track_count = 0
    count = 0
    classID_dict = {0: ("Escooter", (0, 90, 255)), 1: ("Pedestrians", (255, 90, 0)), 2: ("Cyclists", (90, 255, 0))}
    pbar = TqdmExtraFormat(total = total_frames, desc='Inference Progress: ')
    while video_capture.isOpened():
        _, frame = video_capture.read()
        count += 1

        # if count%8 != 0:
        #   continue
        
        if _:
            # Results. Try printing or modifying this array to get different classes
            results = model(frame, size=1920)
            result = results.xyxy[0]
            
            # score_class_id = []
            if len(result) > 0: 
              dets = []
              for items in result:
                dets.append(items[:].tolist())
              
              dets = np.array(dets)
              # print('dets : ', dets)
              tracker = Object_tracker.update(dets)
              
            else:
              tracker = Object_tracker.update()
            # print('tracker : ',tracker)

            img, track_dict, track_count = show_tracker_bbox_2(tracker, frame, track_dict, track_count)
            
            if velocity_frame_window < track_count:
              for i in track_dict.keys():
                if len(track_dict[i])>velocity_frame_window:
                  # prev_point = track_dict[i][track_count - 1 -velocity_frame_window] #  <-- this will give an error becz of deleting values from dict 
                  prev_point = track_dict[i][0]
                  curr_point = track_dict[i][-1]
                  del track_dict[i][0]
                  distance = int(math.sqrt(math.pow(prev_point[0] - curr_point[0], 2) + math.pow(prev_point[1] - curr_point[1], 2)))
                  velo_in_pix_s = int((distance * FPS_OF_RECORDING)/ (velocity_frame_window)) 
                  s_label = f'speed: {int(velo_in_pix_s)} pix/s'
                  cv2.putText(img, s_label, (int(curr_point[0])+15, int(curr_point[1])), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)

            video_output.write(img)
            pbar.update(framecount)
            
        else:
          break

    video_capture.release()
    video_output.release()

# Model
model_weight = r'/content/tl_yolo5l6_78k_bs_3.pt'
input_directory = r'/content/31.mp4'
output_directory = r'/content/vel_result_2.mp4'
class_name = None
video_inference(input_directory, output_directory, model_weight, class_name)



requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...


YOLOv5 🚀 3e24bc1 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)



  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13

requirements: 1 package updated per /content/Traffic_Camera_Tracking/YoloV5/yolo_v5_main_files/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



Fusing layers... 
Model Summary: 476 layers, 76134048 parameters, 0 gradients
Adding AutoShape... 
Inference Progress: 100%|██████████| 1320/1320 [10:37<00:00,  2.11it/s]